## Advent of code 2022 day 11-20
See https://adventofcode.com/

In [ ]:
# note that this notebook requires the .venv-pypy environment for pypy 3.9
# to activate it from a git bash shell: source .venv-pypy/Scripts/activate
# to generate its requirements: pip freeze > .venv-pypy-requirements.txt

import collections
import itertools
import re
import copy
import math
import sys
import time
import json
import heapq
import bisect
import random
import sortedcontainers
#import cProfile

In [ ]:
# utility functions and version check

def get_line_groups(lines, nostrip=False):
    '''return list of lists of lines, each separated by empty lines, ignores empty lines from start and end,
    by default also strips all lines (if nostrip is set only strips empty lines)'''
    lines=list(lines)
    lines.append('') # add terminator
    res=[]
    group=[]
    for line in lines:
        line_str=line.strip()
        if nostrip==False or len(line_str)<1:
            line=line_str
        if len(line)>0:
            group.append(line)
        elif len(group)>0: # close group
            res.append(group)
            group=[]
    return res

class StopExecution(Exception):
    def _render_traceback_(self):
        pass

def exit():
    raise StopExecution()
    
print(f'python version: {sys.version}')
print(f'# start_ts={int(time.time())}') # supports ranking using an honor system, before starting include this line
# in the header of your solution (which should start with a line like # 2019 day 2), then whenever you want save
# a private leaderboard json file, and run python privaterank.py filename.json

In [ ]:
# 2022 day 11
# mv ~/Downloads/input* data_src/2022-day-11-input.txt
# big input file looks like: monkey rules
# idea: part 1 parse ..., then ...

sample2='''
Monkey 0:
  Starting items: 79, 98
  Operation: new = old * 19
  Test: divisible by 23
    If true: throw to monkey 2
    If false: throw to monkey 3

Monkey 1:
  Starting items: 54, 65, 75, 74
  Operation: new = old + 6
  Test: divisible by 19
    If true: throw to monkey 2
    If false: throw to monkey 0

Monkey 2:
  Starting items: 79, 60, 97
  Operation: new = old * old
  Test: divisible by 13
    If true: throw to monkey 1
    If false: throw to monkey 3

Monkey 3:
  Starting items: 74
  Operation: new = old + 3
  Test: divisible by 17
    If true: throw to monkey 0
    If false: throw to monkey 1
'''

class Monkey:
    def __init__(self):
        self.idx=None # id
        self.itemworries=[] # list of item worry levels
        self.op=None # '+' or '*'
        self.opval=None # integer number or 'old'
        self.testval=None # 'divisible by' integer number
        self.truem=None # true monkey number
        self.falsem=None # false monkey number
        self.inspectcount=0

    def parseblock(self, lines, idx):
        self.idx=idx
        s=lines[0].split()
        assert s[0]=='Monkey'
        assert s[1]==str(idx)+':'
        prefix='Starting items: '
        assert lines[1].startswith(prefix)
        for s in re.split(f'[\s,]', lines[1][len(prefix):]):
            if s:
                self.itemworries.append(int(s))
        s=lines[2].split()
        assert s[-3]=='old'
        self.op=s[-2]
        self.opval=s[-1]
        if self.opval!='old':
            self.opval=int(self.opval)
        s=lines[3].split()
        assert s[-2]=='by'
        self.testval=int(s[-1])
        s=lines[4].split()
        assert s[-2]=='monkey'
        self.truem=int(s[-1])
        s=lines[5].split()
        assert s[-2]=='monkey'
        self.falsem=int(s[-1])

    def __repr__(self):
        return f'Monkey {self.idx} (worries: {self.itemworries}, op: {self.op}, opval: {self.opval}, testval: {self.testval}, truem: {self.truem}, falsem: {self.falsem})'

    def do_turn(self, monks, part, modval):
        while len(self.itemworries)>0:
            w=self.itemworries.pop(0)
            opval=w if self.opval=='old' else self.opval
            if self.op=='+':
                w+=opval
            elif self.op=='*':
                w*=opval
            else:
                assert False
            if part==1:
                w//=3
            elif part==2:
                w%=modval
            else:
                assert False
            if w%self.testval==0:
                monks[self.truem].itemworries.append(w)
            else:
                monks[self.falsem].itemworries.append(w)
            self.inspectcount+=1

def do_round(monks, part, modval):
    for m in monks:
        m.do_turn(monks, part, modval)

def lcm(a, b): # least common multiple
    return a*b//math.gcd(a, b)

sample1=open('data_src/2022-day-11-input.txt').read()
groups=get_line_groups(sample1.splitlines(), nostrip=False)
monks=[] # list of Monkey
for i, lines in enumerate(groups):
    m=Monkey()
    m.parseblock(lines, i)
    monks.append(m)
# part 1
for _ in range(20):
    do_round(monks, 1, 0)
inspects=[ m.inspectcount for m in monks ]
inspects.sort()
print(f'part 1: {inspects[-2]*inspects[-1]}')
# part 2
monks=[] # list of Monkey
for i, lines in enumerate(groups):
    m=Monkey()
    m.parseblock(lines, i)
    monks.append(m)
modval=1
for m in monks:
    modval=lcm(modval, m.testval)
print(f'part 2: {modval=}')
for _ in range(10000):
    do_round(monks, 2, modval)
inspects=[ m.inspectcount for m in monks ]
inspects.sort()
print(f'part 2: {inspects[-2]*inspects[-1]}')

# part 1: 113232
# part 2: 29703395016


In [ ]:
# TEMPLATE
# 2022 day 11
# start_ts=RUN FIRST CELL TO GET TIME CODE BEFORE OPENING THE ASSIGNMENT
# mv ~/Downloads/input* data_src/2022-day-11-input.txt
# big input file looks like: 
# idea: part 1 parse ..., then ...

sample2='''

'''

sample1=open('data_src/2022-day-11-input.txt').read()
lines=[s for s in sample2.splitlines() if len(s)>0 ]
groups=get_line_groups(sample2.splitlines(), nostrip=False)
data=[ int(s) for s in lines[0].split(',') ]
data=[ s.split() for s in lines ]
data=[ [cmd, int(num), 0] for cmd, num in data ]
data=[ result.group(1, 2, 3, 4, 5, 6, 7) for s in lines if (result:= re.match(r'(\w+)\s*x=([\d\-]+)\.\.([\d\-]+),y=([\d\-]+)\.\.([\d\-]+),z=([\d\-]+)\.\.([\d\-]+)', s)) ]
data=[ (row[0], int(row[1]), int(row[2]), int(row[3]), int(row[4]), int(row[5]), int(row[6]) ) for row in data ]
# template, remove what's not needed